In [ ]:
# train a NN after getting data, setting up a model
# https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html


In [1]:
# pre-reg code
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [2]:
# hyperparameters
# number of epochs - number of times to iterate over the dataset
# batch size - number of data samples propagated through the network before
#  the parameters are updated
# learning rate - how much to update params at each batch/epoch
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# optimization loop
# each optimization loop is an epoch
#   1. train loop - update params
#   2. validation/test loop - check model perf

# key components
# -loss function - measure dissimilarity of obtained result to a target value 
#  goal of minimizing this (e.g., nn.MSELoss for regression, nn.NLLLoss for classification,
#  nn.CrossEntropyLoss combining nn.LogSoftmax and nn.NLLLoss)
# -optimizer - process for adjusting model parameters to reduce model error using 
#  algorithms such as Stochastic Gradient Descent (SGD), encapsulated in optimizer 
#  object. Requires the model parameters and learning rate as inputs
# Step 1: optimizer.zero_grad()
# Step 2: backprop loss - loss.backward()
# Step 3: use grads in optimizer.step() and adjust model params

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [4]:
# run the training process
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.292907  [   64/60000]
loss: 2.284126  [ 6464/60000]
loss: 2.265612  [12864/60000]
loss: 2.260755  [19264/60000]
loss: 2.235880  [25664/60000]
loss: 2.215730  [32064/60000]
loss: 2.210976  [38464/60000]
loss: 2.182745  [44864/60000]
loss: 2.182799  [51264/60000]
loss: 2.145407  [57664/60000]
Test Error: 
 Accuracy: 52.0%, Avg loss: 2.143777 

Epoch 2
-------------------------------
loss: 2.150179  [   64/60000]
loss: 2.139690  [ 6464/60000]
loss: 2.078927  [12864/60000]
loss: 2.095026  [19264/60000]
loss: 2.031178  [25664/60000]
loss: 1.983102  [32064/60000]
loss: 1.995885  [38464/60000]
loss: 1.921708  [44864/60000]
loss: 1.926623  [51264/60000]
loss: 1.845320  [57664/60000]
Test Error: 
 Accuracy: 57.3%, Avg loss: 1.850022 

Epoch 3
-------------------------------
loss: 1.880700  [   64/60000]
loss: 1.846945  [ 6464/60000]
loss: 1.729359  [12864/60000]
loss: 1.768213  [19264/60000]
loss: 1.661521  [25664/60000]
loss: 1.619826  [32064/600